Calculate the global mean T2m for 1979-2014 and 2070-2099 of the CMIP5 models.  Outputs both the global mean T2m of the ensemble mean and the first ensemble member.  Concatenates 1979-2005 of the historical simulations with 2005-2014 of the RCP8.5 simulations and uses 2070-2099 of the RCP8.5 simulations.  This code works on a CMIP archive that is internal to the Climate and Global Dynamics Laboratory, NCAR.  I you want to run this on your own archive, you'd need to modify histpath and rcp85path and make sure the directory structure is set up in the right way.

In [8]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
from numpy import nan
import sys
import warnings

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import calendar_utils as cal

importlib.reload(read)
importlib.reload(avg)

warnings.filterwarnings('ignore')

Set path for CMIP5 models (historical and RCP8.5) and output directory

In [2]:
histpath="/project/cmip5/historical/Amon/"
rcp85path="/project/cmip5/rcp85/Amon/"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/SHJET/DATA/"

Information on the models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of member and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [3]:
cmip5models=pd.read_csv('../cmip5csvinfo.csv')

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for Past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

This is the main part of the script.  It loops over models and ensemble members and calculates the global mean T2m for JJA for both the ensemble mean and the first member.  These are then output to netcdf

In [17]:
models=cmip5models['Model']
nmodels=models.size

tas1memp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="tas1memp")
tas1memf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="tas1memf")
tasemp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="tasemp")
tasemf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="tasemf")

# loop over models
for index, modname in models.iteritems():
    
    # ----- sort out the PAST period ---
    nmems = cmip5models.loc[index, "Nmempast"]
    
    for imem in range(1,nmems+1,1):
        memstr="r"+str(imem)+"i1p1"
        # check if a special order is needed
        changeorder = cmip5models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir = histpath+"tas/"+modname+"/"+memstr+"/"
        rcp85dir = rcp85path+"tas/"+modname+"/"+memstr+"/"


        # read in zonal tas, concatenate historical to 2005 and rcp8.5 to 2014
        tashist=read.read_sfc(histdir+"*.nc", datebegp, "2005-12")
        tasrcp=read.read_sfc(rcp85dir+"*.nc", "2006-01", dateendp)
        tas=xr.concat([tashist, tasrcp], dim="time", join="override")
        
        # check the array length.  If this fails, try ending historical in 2005-11
        # and beginning rcp in 2006-01 to fix Hadley Center.
        if (tas.time.size != nmonthsp):
            print("switching to the assumption that the historical ends in 2006-11")
            tashist=read.read_sfc(histdir+"*.nc", datebegp, "2005-11")
            tasrcp=read.read_sfc(rcp85dir+"*.nc", "2005-12", dateendp)
            tas=xr.concat([tashist, tasrcp], dim="time", join="override")
            
        # check again
        if (tas.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but tas has size "+str(tas.time.size))
            
        # calculate the JJA mean
        tasjja = cal.season_mean(tas,"tas", season="JJA")
        tasgm = avg.cosweightlonlat(tasjja,0,360,-90,90)

        # save the first member
        if (imem == 1):
            tas1memp[index]=tasgm

        # calculate the ensemble mean
        tasemp[index]=tasemp[index]+tasgm/float(nmems)
        
    # ---sort out the future period
    nmems = cmip5models.loc[index, "Nmemfuture"]
    for imem in range(1,nmems+1,1):
        memstr="r"+str(imem)+"i1p1"
        # check if a special order is needed
        changeorder = cmip5models.loc[index, "specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing future for "+modname+" "+memstr+"...")
        rcp85dir = rcp85path+"tas/"+modname+"/"+memstr+"/"
        
        tas = read.read_sfc(rcp85dir+"*.nc",datebegf, dateendf)
        
        if (tas.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but tas has size "+str(tas.time.size))
            
        tasjja = cal.season_mean(tas,"tas", season="JJA")
        tasgm = avg.cosweightlonlat(tasjja, 0, 360, -90, 90)
        
        # save the first member
        if (imem == 1):
            tas1memf[index]=tasgm
            
        # calculate the ensemble mean
        tasemf[index]=tasemf[index]+tasgm/float(nmems)    

Processing past for ACCESS1-0 r1i1p1...
Processing future for ACCESS1-0 r1i1p1...
Processing past for ACCESS1-3 r1i1p1...
Processing future for ACCESS1-3 r1i1p1...
Processing past for bcc-csm1-1 r1i1p1...
Processing future for bcc-csm1-1 r1i1p1...
Processing past for bcc-csm1-1-m r1i1p1...
Processing future for bcc-csm1-1-m r1i1p1...
Processing past for BNU-ESM r1i1p1...
Processing future for BNU-ESM r1i1p1...
Processing past for CanESM2 r1i1p1...
Processing past for CanESM2 r2i1p1...
Processing past for CanESM2 r3i1p1...
Processing past for CanESM2 r4i1p1...
Processing past for CanESM2 r5i1p1...
Processing future for CanESM2 r1i1p1...
Processing future for CanESM2 r2i1p1...
Processing future for CanESM2 r3i1p1...
Processing future for CanESM2 r4i1p1...
Processing future for CanESM2 r5i1p1...
Processing past for CCSM4 r1i1p1...
Processing past for CCSM4 r2i1p1...
Processing past for CCSM4 r3i1p1...
Processing past for CCSM4 r4i1p1...
Processing past for CCSM4 r5i1p1...
Processing past 

In [18]:
tas1memp.to_netcdf(path=pathout+"globalmeantascmip5.nc")
tas1memf.to_netcdf(path=pathout+"globalmeantascmip5.nc", mode="a")
tasemp.to_netcdf(path=pathout+"globalmeantascmip5.nc", mode="a")
tasemf.to_netcdf(path=pathout+"globalmeantascmip5.nc", mode="a")